# Scripts

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def obtener_contenido_pagina(url):
    response = requests.get(url)
    return response.content

In [3]:
def analizar_contenido_html(html):
    return BeautifulSoup(html,"html.parser")

In [4]:
url = "https://www.cyberpuerta.mx/Promociones/"
soup = BeautifulSoup(url.text,"html.parser")

AttributeError: 'str' object has no attribute 'text'

In [16]:
data = []

def procesar_pagina(soup):
    titulos = []
    precios_originales = []
    precios_descuento = []
    precios_envio = []
    skus = []
    stocks = []

    titulo_items = soup.find_all("div",class_="emproduct_right")
    
    for item in titulo_items:
        titulo = item.a.text.strip()
        titulos.append(titulo)

    precio_original_items = soup.find_all("span", class_="oldPrice")

    for item in precio_original_items:
        precio_original = item.text.strip()
        precios_originales.append(precio_original)

    precio_descuento_items = soup.find_all("div", class_="emproduct_right_price_left")

    for item in precio_descuento_items:
        precio_descuento = item.label.text.strip()
        precios_descuento.append(precio_descuento)

    sku_items = soup.find_all("div", class_="emproduct_right_artnum")

    for item in sku_items:
        sku = item.text.strip()
        skus.append(sku)

    precio_envio_items = soup.find_all("span",class_={"deliveryvalues","deliverytextfree"})

    for item in precio_envio_items:
        precio_envio = item.text
        precios_envio.append(precio_envio)

    stocks_items = soup.find_all("div", class_="emstock")

    for item in stocks_items:
        stock = item.text.strip()
        stocks.append(stock[1])

    
    min_lenght = min(len(titulos),len(precios_originales),len(precios_descuento),len(precios_envio),len(skus),len(stocks))
    
    for i in range(min_lenght):
        data.append({
            "Titulo":titulos[i] if i< len(titulos) else None,
            "SKU":skus[i] if i< len(skus) else None,
            "Precio original":precios_originales[i] if i< len(precios_originales) else None,
            "Precio descuento":precios_descuento[i] if i< len(precios_descuento) else None,
            "Precio Envio":precios_envio[i] if i< len(precios_envio) else None,
            "Stock": stocks[i] if i< len(stocks) else None})




In [18]:
def manejar_paginacion(url_base,num_paginas):

    #url_base = "https://www.cyberpuerta.mx/Promociones/"

    for i in range(1,num_paginas + 1):
        url = url_base +str(i)+"/"
        contenido_pagina = obtener_contenido_pagina(url)
        soup = analizar_contenido_html(contenido_pagina)
        procesar_pagina(soup)



In [19]:
url_base = "https://www.cyberpuerta.mx/Promociones/"
num_paginas = 10
manejar_paginacion(url_base,num_paginas)

In [21]:
import pandas as pd
df = pd.DataFrame(data)
df

,Titulo,SKU,Precio original,Precio descuento,Precio Envio,Stock
0,"Memoria RAM HPE DDR4, 2933MHz, 16GB, CL21, Dua...",SKU: P00922-B21,"$13,644.00","$3,929.00",Envío gratis,i
1,"Vorago Silla Gamer CGC500, hasta 120kg, Negro/...",SKU: CGC-500/BL,"$3,641.00","$2,949.00",Envío gratis,i


In [22]:
import datetime
fecha_actual = datetime.datetime.now().strftime("%d-%m-%Y")

df.to_csv(f"./dataset/Cyberpuerta-WebScrap-{fecha_actual}.csv",index=False)